In [1]:
import pandas as pd

In [2]:
df0 = pd.read_csv('IMDB-Movie-Data.csv')
df = df0.copy()

#### Q1. Top‐3 movies with the highest ratings in 2016? 

In [3]:
df1 = df[df['Year'] == 2016]
df1_1 = df1.nlargest(3,'Rating')
df1_2 = df1[df1['Rating'] == df1_1['Rating'].min()]

pd.concat([df1_1, df1_2]).drop_duplicates()

,Rank,Title,Genre,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
2,118,Dangal,Action|Biography|Drama,Nitesh Tiwari,Aamir Khan| Sakshi Tanwar| Fatima Sana Shaikh|...,2016,161,8.8,48969,11.15,NaN
4,97,Kimi no na wa,Animation|Drama|Fantasy,Makoto Shinkai,Ryunosuke Kamiki| Mone Kamishiraishi| Ryo Nari...,2016,106,8.6,34110,4.68,79.0
15,862,Koe no katachi,Animation|Drama|Romance,Naoko Yamada,Miyu Irino| Saori Hayami| Aoi Yuki| Kensho Ono,2016,129,8.4,2421,NaN,80.0


#### Q2. The actor generating the highest average revenue?

In [4]:
#deal with Actors (from A|B|C to [A, B, C])
for i in range(len(df['Actors'])):
    actors = df['Actors'][i].split("|")
    for j in range(len(actors)):
        actors[j] = actors[j].strip()
    df['Actors'][i] = actors    

#find all actors and compute actor's average revenue
Actors = []
ave_revenue = []

for i in range(len(df['Title'])):
    for j in range(len(df['Actors'][i])):
        actor = df['Actors'][i][j]
        if actor in Actors:
            continue
        Actors.append(actor)
        
        actor_revenue = []
        for k in range(len(df['Title'])):
            if actor in df['Actors'][k] and df['Revenue (Millions)'][k] != '':
                actor_revenue.append(float(df['Revenue (Millions)'][k]))
        
        if len(actor_revenue) == 0:
            ave_revenue.append(0)
            continue
        ave_revenue.append(round(sum(actor_revenue)/len(actor_revenue), 2))
        
#create a new Dataframe        
df2 = pd.DataFrame({'Actor': Actors, 
                      'average revenue': ave_revenue}).fillna(0)

df2_1 = df2.nlargest(3,'average revenue')
df2_2 = df2[df2['average revenue'] == df2_1['average revenue'].min()]

pd.concat([df2_1, df2_2]).drop_duplicates()

<ipython-input-4-bf2cbfeae43d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Actors'][i] = actors


,Actor,average revenue
126,Daisy Ridley,936.63
127,John Boyega,936.63
276,Alan Tudyk,532.17
277,Donnie Yen,532.17


#### Q3. The average rating of Emma Watson’s movies?

In [5]:
EW_join = []
for i in range(len(df['Actors'])):
    if 'Emma Watson' in df['Actors'][i]:
        EW_join.append(i)

df['Rating'].iloc[EW_join].mean()

7.175

#### Q4. Top‐3 directors who collaborate with the most actors? 

In [6]:
Directors = []
number_of_director_collaborate = []
for i in range(len(df['Director'])):
    director = df['Director'][i]
    if director in Directors:
        continue
    Directors.append(director)
    
    movie_of_director = df[df['Director'] == director]
    collaborate = []
    for j in range(len(movie_of_director['Actors'])):
        for k in range(len(movie_of_director['Actors'].iloc[j])):
            actor = movie_of_director['Actors'].iloc[j][k]
            if actor in collaborate:
                continue
            collaborate.append(actor)
    
    number_of_director_collaborate.append(len(collaborate))
    
    
df4 = pd.DataFrame({'Director': Directors, 
                      'collaborate number': number_of_director_collaborate}).fillna(0)

df4_1 = df4.nlargest(3,'collaborate number')
df4_2 = df4[df4['collaborate number'] == df4_1['collaborate number'].min()]

pd.concat([df4_1, df4_2]).drop_duplicates()

,Director,collaborate number
49,Ridley Scott,28
191,M. Night Shyamalan,24
59,Danny Boyle,20
445,Paul W.S. Anderson,20


#### Q5. Top‐2 actors playing in the most genres of movies?

In [7]:
#deal with Genre (from A|B|C to [A, B, C])
for i in range(len(df['Genre'])):
    df['Genre'][i] = df['Genre'][i].split("|") 
    
number_of_genre = []
for m in range(len(Actors)):
    actor_join_iloc = []
    for i in range(len(df['Actors'])):
        if Actors[m] in df['Actors'][i]:
            actor_join_iloc.append(i)
        
    actor_join = df.iloc[actor_join_iloc]
    genres = []
    for j in range(len(actor_join['Genre'])):
        for k in range(len(actor_join['Genre'].iloc[j])):
            genre = actor_join['Genre'].iloc[j][k]
            if genre in genres:
                continue
            genres.append(genre) 
    number_of_genre.append(len(genres))
    
    
df5 = pd.DataFrame({'Actor': Actors, 
                      'number of genre': number_of_genre}).fillna(0)

df5_1 = df5.nlargest(2,'number of genre')
df5_2 = df5[df5['number of genre'] == df5_1['number of genre'].min()]

pd.concat([df5_1, df5_2]).drop_duplicates()


<ipython-input-7-4d90732f5a7d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Genre'][i] = df['Genre'][i].split("|")


,Actor,number of genre
62,Brad Pitt,14
24,Hugh Jackman,13
25,Scarlett Johansson,13
212,Amy Adams,13
465,Chloe Grace Moretz,13
715,Johnny Depp,13


#### Q6. Top‐3 actors whose movies lead to the largest maximum gap of years?

In [8]:
gap_of_year = []
for m in range(len(Actors)):
    actor_join_iloc = []
    for i in range(len(df['Actors'])):
        if Actors[m] in df['Actors'][i]:
            actor_join_iloc.append(i)
        
    actor_join = df.iloc[actor_join_iloc]
    gap_of_year.append(actor_join['Year'].max() - actor_join['Year'].min())
    
df6 = pd.DataFrame({'Actor': Actors, 
                      'gap of years': gap_of_year}).fillna(0)

df6_1 = df6.nlargest(3,'gap of years')
df6_2 = df6[df6['gap of years'] == df6_1['gap of years'].min()]

pd.concat([df6_1, df6_2]).drop_duplicates()


,Actor,gap of years
0,Christian Bale,10
13,Anne Hathaway,10
24,Hugh Jackman,10
25,Scarlett Johansson,10
26,Matt Damon,10
28,Mark Wahlberg,10
62,Brad Pitt,10
77,Christopher Plummer,10
78,Tom Hanks,10
148,Bryce Dallas Howard,10


#### Q7. Find all actors who collaborate with Johnny Depp in direct and indirect ways 

In [9]:
had_collaborate = ['Johnny Depp']
movie_index = []
finish = False
while finish == False:
    start_len = len(had_collaborate)
    for i in range(len(df['Actors'])):
        if i in movie_index:
            continue
        for j in range(len(df['Actors'][i])):
            if df['Actors'][i][j] in had_collaborate:
                movie_index.append(i)
                
    for m in movie_index:
        for n in range(len(df['Actors'][m])):
            if df['Actors'][m][n] in had_collaborate:
                continue
            
            had_collaborate.append(df['Actors'][m][n])
            
    if len(had_collaborate) == start_len:
        finish = True

df7 = pd.DataFrame({'Actors who collaborate with Johnny Depp in direct and indirect ways ': had_collaborate[1:]})
df7

,Actors who collaborate with Johnny Depp in direct and indirect ways
0,Helena Bonham Carter
1,Alan Rickman
2,Timothy Spall
3,Orlando Bloom
4,Keira Knightley
...,...
1569,Michael Constantine
1570,Lainie Kazan
1571,Josh Stewart
1572,Bill Engvall
